In [ ]:
import keras

model = keras.models.load_model("../Data/convnet_from_scratch_with_augmentation.keras")
model.summary()

In [17]:
import numpy as np
import matplotlib.pyplot as plt

img_path = keras.utils.get_file(fname="cat.jpg",
                                origin="https://img-datasets.s3.amazonaws.com/cat.jpg")

In [ ]:

plt.imshow(plt.imread(img_path))

In [19]:
from keras import layers

layers_output = []
layer_names = []

for layer in model.layers:
    if isinstance(layer, (layers.Conv2D, layers.MaxPooling2D)):
        layers_output.append(layer.output)
        layer_names.append(layer.name)
activation_model = keras.Model(inputs=model.input, outputs=layers_output)

In [20]:
def get_img_array(img_path, target_size):
    img = keras.utils.load_img(img_path, target_size=target_size)
    array = keras.utils.img_to_array(img)
    array = np.expand_dims(array, axis=0)
    return array

In [ ]:
img_tensor = get_img_array(img_path, target_size=(180,180))
activations = activation_model.predict(img_tensor)

In [ ]:
first_layer_activation = activations[0]
print(first_layer_activation.shape)

In [ ]:
plt.matshow(first_layer_activation[0, :, :, 4], cmap="viridis")

In [ ]:
images_per_row = 16
for layer_name, layers_activation in zip(layer_names, activations):
    n_features = layers_activation.shape[-1]
    size = layers_activation.shape[1]
    n_cols = n_features // images_per_row
    display_grid = np.zeros(((size+1) *n_cols-1,
                            images_per_row*(size+1) -1))
    for col in range(n_cols):
        for row in range(images_per_row):
            channel_index = col * images_per_row + row
            channel_image = layers_activation[0, :, :, channel_index].copy()
            if channel_image.sum() != 0:
                channel_image -= channel_image.mean()
                channel_image /= channel_image.std()
                channel_image *= 64
                channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype("uint8")
            display_grid[
                col * (size+1):(col+1)*size+col,
                row * (size+1): (row+1)*size+row] = channel_image
    scale = 1./size
    plt.figure(figsize=(scale*display_grid.shape[1],
                        scale*display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.axis("off")
    plt.imshow(display_grid, aspect="auto", cmap="viridis")

In [ ]:
model = keras.applications.xception.Xception(weights="imagenet", include_top=False)
model.summary()